# Retry Query Engine

Uncomment to add your OpenAI API key or enable debugging

In [1]:
import os
# os.environ['OPENAI_API_KEY'] = "INSERT OPENAI KEY"

# import logging
# import sys

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

First we ingest the document.

In [2]:
from llama_index.indices.vector_store.base import VectorStoreIndex
from llama_index.query_engine.retry_source_query_engine import RetrySourceQueryEngine
from llama_index.readers.file.base import SimpleDirectoryReader


In [3]:
documents = SimpleDirectoryReader('../data/paul_graham/').load_data()
index = VectorStoreIndex.from_documents(documents)
query = "What did the author do growing up?"

We will query with the default settings first.

In [4]:
# Default query engine
default_query_engine = index.as_query_engine()
response = default_query_engine.query(query)
print(f"Default query engine response: {response}")

Default query engine response: 
The author grew up writing essays, learning Italian, exploring Florence, painting people, working with computers, attending RISD, living in a rent-stabilized apartment, building an online store builder, editing Lisp expressions, publishing essays online, writing essays, painting still life, working on spam filters, cooking for groups, and buying a building in Cambridge.


Now with retry, if the response does not pass the evaluator the query engine passed in will be rerun with the negative counterexample.

In [5]:
from llama_index.query_engine.retry_query_engine import (
    RetryQueryEngine,
)
from llama_index.evaluation.base import QueryResponseEvaluator

# Query engine with retry
query_response_evaluator = QueryResponseEvaluator()
retry_query_engine = RetryQueryEngine(index.as_query_engine(), query_response_evaluator)
retry_response = retry_query_engine.query(query)
print(f"Query engine with retry response: {retry_response}")

Query engine with retry response: 
The author grew up writing essays, learning Italian, exploring Florence, painting people, working with computers, attending RISD, living in a rent-controlled apartment, building an online store builder, editing code, launching software, publishing essays online, writing essays, painting still life, working on spam filters, cooking for groups, and buying a building in Cambridge.


The Source Retry modifies the query source nodes by filtering the existing source nodes for the query based on llm node evaluation.

A better solution would be to re-retrieve based on filtering out the nodes that didn't pass evaluation.

TODO

In [6]:
retry_source_query_engine = RetrySourceQueryEngine(index.as_query_engine(), query_response_evaluator)
retry_source_response = retry_source_query_engine.query(query)
print(f"Query engine with retry source response: {retry_source_response}") 

Query engine with retry source response: 
The author grew up writing essays, learning Italian, exploring Florence, painting people, learning about computers, attending RISD, living in a rent-stabilized apartment, building an online store builder, editing Lisp expressions, publishing essays online, writing essays, painting still life, working on spam filters, cooking for groups, and buying a building in Cambridge.


This module tries to use guidelines to direct the evaluator's behavior. You can customize your own guidelines.

In [7]:
from llama_index.evaluation.guideline_eval import GuidelineEvaluator, DEFAULT_GUIDELINES
from llama_index.response.schema import Response
from llama_index.indices.query.query_transform.feedback_transform import (
    FeedbackQueryTransformation,
)
from llama_index.query_engine.retry_query_engine import (
    RetryGuidelineQueryEngine,
)

# Guideline eval
guideline_eval = GuidelineEvaluator(
    guidelines=DEFAULT_GUIDELINES
    + "\nThe response should not be overly long.\n"
    "The response should try to summarize where possible.\n"
)  # just for example
typed_response = response if isinstance(response, Response) else response.get_response()
eval = guideline_eval.evaluate_response(query, typed_response)
print(f"Guideline eval evaluation result: {eval.feedback}")
feedback_query_transform = FeedbackQueryTransformation(resynthesize_query=True)
transformed_query = feedback_query_transform.run(query, {"evaluation": eval})
print(f"Transformed query: {transformed_query.query_str}")
retry_guideline_query_engine = RetryGuidelineQueryEngine(
    index.as_query_engine(), guideline_eval, resynthesize_query=True
)
retry_guideline_response = retry_guideline_query_engine.query(query)
print(f"Query engine with retry guideline response: {retry_guideline_response}")

Guideline eval evaluation result: The response is too long and should be summarized. It should also include specific numbers or statistics when possible.
Transformed query: Here is a previous bad answer.

The author grew up writing essays, learning Italian, exploring Florence, painting people, working with computers, attending RISD, living in a rent-stabilized apartment, building an online store builder, editing Lisp expressions, publishing essays online, writing essays, painting still life, working on spam filters, cooking for groups, and buying a building in Cambridge.
Here is some feedback from the evaluator about the response given.
The response is too long and should be summarized. It should also include specific numbers or statistics when possible.
Now answer the question.

What experiences did the author have growing up?
Query engine with retry guideline response: 
The author had a wide range of experiences growing up. They wrote essays, explored Florence, painted people, worked